# STAT 440 Statistical Data Management - Fall 2021
## Week04 Lab01 Mon
### Only the driver should submit this assignment into their individual student repo. Save the filename as lab01-week04-mon-netID.Rmd. Then render this file to .html when completed. 
#### This lab must be submitted within 10 minutes of the end of class time. Any lab submitted after 11:00 am will be deducted 1 point for being late.

**#1** Using Markdown syntax, make a numbered list with the first and last names of everyone at your station followed by a dash then the laptop group the everyone belongs; for example, Chris Kinson - D. Your name should appear in bold text. The possible laptop groups are: 

A. MacBook with a Mini Display Port but not USB-C  
B. MacBook with a USB-C  
C. Windows laptop without an HDMI port  
D. Laptop does not fit into A, B, or C.  

Answers will vary, but must be a proper list with names and letters. Driver's name must be bold text.

**#2** Import the Rental Inspection Grades Listing Data using one programming language software and the data URL https://github-dev.cs.illinois.edu/stat440-fa21/stat440-fa21-course-content/raw/master/data/rental-inspections-grades-data05.json or Box data URL https://uofi.box.com/shared/static/at5sydtqaals7opqlq35p59xu0f9lsyx.json. Make sure that the Inspection Date and Expiration Date columns are formatted as dates or datetimes; not character format. Now, print the structure of the data. **This structure should match the data description below.** 

  - The dataset (a .json file) contains the 1730 rental properties in Urbana with 7 variables ("columns") per property which includes details about the parcel number, address, grade, inspection date, and more. The rental properties are inspected on a rotating basis and can receive a letter grade of A, B, C, D, F, and N. The best grade is A. The original source is the City of Urbana.


In [5]:
import pandas as pd
import urllib.request, json 
with urllib.request.urlopen("https://uofi.box.com/shared/static/at5sydtqaals7opqlq35p59xu0f9lsyx.json") as url0:
    rentals = pd.DataFrame(json.loads(url0.read().decode()))

rentals['Expiration Date'] = pd.to_datetime(rentals['Expiration Date'])
rentals['Inspection Date'] = pd.to_datetime(rentals['Inspection Date'])
rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730 entries, 0 to 1729
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Property Address  1730 non-null   object        
 1   Parcel Number     1730 non-null   int64         
 2   Inspection Date   1730 non-null   datetime64[ns]
 3   Grade             1730 non-null   object        
 4   License Status    1730 non-null   object        
 5   Expiration Date   1456 non-null   datetime64[ns]
 6   Mappable Address  1730 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 94.7+ KB


**#3** Web scrape the Champaign County Assessor's [website](https://champaignil.devnetwedge.com/) to extract a property's Net Taxable Value in US Dollars. Do this for the 10th, 20th, 30th, 40th, and 50th properties in the dataset from Problem **#2** and print these 5 values.


In [6]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
root_url = 'https://champaignil.devnetwedge.com/parcel/view/'
net_tax02 = ['']*len(rentals)
for i in range(9,59,10):
    url = root_url + str(rentals['Parcel Number'].iloc[i]) + '/2020'
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    net_tax01 = soup.select('tr:nth-child(5) td:nth-child(1) .inner-value')
    for oN in net_tax01:
        net_tax02[i] = [taxes01.text for taxes01 in net_tax01]
net_tax03 = []
for item in net_tax02:
    net_tax03 += item
print(net_tax03)

['81,950', '214,920', '80,980', '23,540', '46,230']


**#4** Import Market at the Square Vendor Products Data using one programming language software and the data URL https://data.urbanaillinois.us/api/views/6gtk-bwms/rows.csv?accessType=DOWNLOAD. No need to consider tokenized URLs. Now, print the structure of the data. **This structure should match the data description below.** 

  - The dataset (a .csv file) contains 1185 rows and 15 columns. The Market at the Square is a seasonal farmer's market where vendors sell vegetables, fruits, meat, and other products. Some vendors may sell non-food items such as arts and crafts. Other vendors are there to promote their organization such as political orgs, candidates, and religious groups. The data contains a listing of products sold by different vendors and with month-by-month details on which products are available, where the availability is Yes or No.


In [8]:
market = pd.read_csv('https://data.urbanaillinois.us/api/views/6gtk-bwms/rows.csv?accessType=DOWNLOAD')
market.info

<bound method DataFrame.info of           Product                             Company  Varieties January  \
0     Accessories                    Andy's Creations         30      No   
1     Accessories          Champaign County Democrats          1      No   
2     Accessories  Champaign County Libertarian Party          1      No   
3     Accessories                CU Immigration Forum          1      No   
4     Accessories       Illinois Stewardship Alliance          1      No   
...           ...                                 ...        ...     ...   
1180     Zucchini                 Kleiss Produce Farm          1      No   
1181     Zucchini                       Meyer Produce          2      No   
1182     Zucchini                      Riverbend Farm          3      No   
1183     Zucchini                    Sola Gratia Farm          2      No   
1184     Zucchini                      Tomahnous Farm          2      No   

     February March April May June July August Septembe

**#5** Reshape the imported data in Problem **4** such that the columns named as the 12 months of the year become one single column named "Months" with their corresponding Yes or No values named as "Availability". Print the first 10 rows of the reshaped data.


In [11]:
import numpy as np
stack_cols = ['January','February','March','April','May','June','July','August','September','October','November','December']
rem_cols = [x for x in market if x not in stack_cols]
stack_df = market[stack_cols].stack(dropna=False).to_frame()
stack_df = stack_df.reset_index()
stack_df.columns = ['newid','Months','Availability']
market02 = pd.merge(market[rem_cols],stack_df,left_on = market.index,right_on='newid',how='right')
market03 = market02.drop(columns=['newid'])
market03.head(10)

,Product,Company,Varieties,Months,Availability
0,Accessories,Andy's Creations,30,January,No
1,Accessories,Andy's Creations,30,February,No
2,Accessories,Andy's Creations,30,March,No
3,Accessories,Andy's Creations,30,April,No
4,Accessories,Andy's Creations,30,May,No
5,Accessories,Andy's Creations,30,June,No
6,Accessories,Andy's Creations,30,July,No
7,Accessories,Andy's Creations,30,August,No
8,Accessories,Andy's Creations,30,September,No
9,Accessories,Andy's Creations,30,October,No
